<a href="https://colab.research.google.com/github/tgeral68/TP-1-chatbot/blob/main/2-AnswerGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from matplotlib import pyplot as plt
from collections import Counter

In [3]:

from datasets import load_dataset
dataset = load_dataset("multi_woz_v22", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/8437 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

mybpe = AutoTokenizer.from_pretrained("openai-community/gpt2")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
from torch.utils.data import Dataset
class WoZWindowedGenerationDataset:
    def __init__(self, dataset, window_size=3):
        self.dataset = dataset
        self.window_size = window_size
        self.index = []
        for i, dial in enumerate(dataset):
            for j, speaker in enumerate(dial['turns']['speaker']):
                if speaker == 1:
                    self.index.append((i,j))
    def __len__(self):
        return len(self.index)

    def __getitem__(self, index):
        i, j = self.index[index]
        dial = self.dataset[i]['turns']['utterance']
        history = dial[max(j-self.window_size, 0):j]
        answer = dial[j]
        return {'history': history,
                'answer': answer}
class DialogueCollator(Dataset):
    def __init__(self, tokenizer, pad_value=mybpe.vocab):
        self.tokenizer = tokenizer
        self.tokenizer.add_special_tokens({'pad_token': '[PAD]', 'sep_token': '[sep]'})
        self.pad_value = self.tokenizer.pad_token_id

    def __call__(self, data):
        history = self.tokenizer([" [sep]".join(d['history']) for d in data],
                                 return_tensors='pt', return_length=True, padding=True)
        answer =  self.tokenizer(["start " + d['answer'] for d in data],
                                 return_tensors='pt', return_length=True, padding=True)

        return {
            'history': history.input_ids,
            'history_len': history.length,
            'answer': answer.input_ids,
            'answer_len': answer.length
        }


In [6]:
training_set = WoZWindowedGenerationDataset(dataset['train'])
collator = DialogueCollator(mybpe)
embeddings = nn.Embedding(len(mybpe.vocab) + 1 , 300, padding_idx=len(mybpe.vocab))

In [7]:

class Seq2SeqRNN(nn.Module):
    def __init__(self,
            input_size=50,
            hidden_size=200,
            output_vocab_size=17,
            num_layers=1,
            bidirectional=False
        ):
        super().__init__()
        self.D = 2 if bidirectional else 1

        self.encoder = nn.LSTM(input_size, hidden_size, num_layers=num_layers,
                           	   batch_first=True, bidirectional=bidirectional)
        self.decoder = nn.LSTM(input_size, hidden_size, num_layers=num_layers,
                           	   batch_first=True, bidirectional=False)
        self.vocab_classifier = nn.Linear(self.D * hidden_size, output_vocab_size)
        self.h_init = nn.Parameter(torch.randn(num_layers  * self.D, hidden_size))
        self.c_init = nn.Parameter(torch.randn(num_layers * self.D, hidden_size))

    ''' RNN forward
        input : dict
            batched dat
    '''
    def forward(self, input_embeds, output_embeds, input_seq_len, output_seq_len, mask=None):
        ### ensure that no grad on the padded part of the sequences
        N, S = input_embeds.shape[0], input_embeds.shape[1]
        input_seq_len = input_seq_len.cpu()
        output_seq_len = output_seq_len.cpu()
        x = torch.nn.utils.rnn.pack_padded_sequence(input_embeds, input_seq_len, batch_first=True, enforce_sorted=False)

        ### forward through the rnn
        x_out, (h_out, c_out) = self.encoder(x,
                                    (
                                    self.h_init.unsqueeze(1).repeat(1, N, 1),
                                    self.c_init.unsqueeze(1).repeat(1, N, 1)
                                    )
                               )


        x_out, _ = torch.nn.utils.rnn.pad_packed_sequence(x_out, batch_first=True)
        ### get last ouput

        y = torch.nn.utils.rnn.pack_padded_sequence(output_embeds, output_seq_len, batch_first=True, enforce_sorted=False)

        y_out, (h_out, c_out) = self.decoder(y,
                                    (
                                    h_out,
                                    c_out
                                    )
                               )
        y_out, _ = torch.nn.utils.rnn.pad_packed_sequence(y_out, batch_first=True)


        return {"output": y_out}

    def encode(self, input_embeds, input_seq_len):
        ### ensure that no grad on the padded part of the sequences
        N, S = input_embeds.shape[0], input_embeds.shape[1]
        input_seq_len = input_seq_len.cpu()
        x = torch.nn.utils.rnn.pack_padded_sequence(input_embeds, input_seq_len, batch_first=True, enforce_sorted=False)

        ### forward through the rnn
        x_out, (h_out, c_out) = self.encoder(x,
                                    (
                                    self.h_init.unsqueeze(1).repeat(1, N, 1),
                                    self.c_init.unsqueeze(1).repeat(1, N, 1)
                                    )
                               )
        x_out, _ = torch.nn.utils.rnn.pad_packed_sequence(x_out, batch_first=True)
        return h_out, c_out

    def decode(self, input_embed, h_in, c_in):
        y = torch.nn.utils.rnn.pack_padded_sequence(input_embed, torch.ones(len(input_embed)), batch_first=True, enforce_sorted=False)

        y_out, (h_out, c_out) = self.decoder(y,
                                    (
                                    h_in,
                                    c_in
                                    )
                               )
        y_out, _ = torch.nn.utils.rnn.pad_packed_sequence(y_out, batch_first=True)

        return y_out, (h_out, c_out)



In [8]:
class MyModel(nn.Module):
    def __init__(self, embedding):
        super().__init__()
        self.embedding = embedding
        self.model = Seq2SeqRNN(input_size=300)
        self.classification_head = nn.Linear(200, embedding.weight.shape[0])
    def forward(self, data):
        output = self.model(
              self.embedding(data['history']),
              self.embedding(data['answer']),
              data['history_len'],
              data['answer_len'])
        y = output["output"]
        return self.classification_head(y)


    def generate(self, data, max_size=100, init_token=0):
        with torch.no_grad():
            generated_tokens = []
            encoder_output_h, encoder_output_c =\
                self.model.encode(self.embedding(data['history']), data['history_len'])
            input_embed = self.embedding(torch.LongTensor([[init_token]]))
            for i in range(max_size):
                y_out, (encoder_output_h, encoder_output_c) = self.model.decode(input_embed, encoder_output_h, encoder_output_c)
                token = self.classification_head(y_out).argmax()
                generated_tokens.append(token.item())
                input_embed = self.embedding(token).unsqueeze(0).unsqueeze(0)
        return generated_tokens

In [17]:
from tqdm.notebook import trange, tqdm
from torch import optim
from torch import nn


class Trainer():
    def __init__(self, model, padding_idx=100):
        self.model = model
        self.padding_idx = padding_idx
        self.optimizer = None

    def before_training_loop(self, learning_rate = 1e-3):
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
        self.criterion = nn.CrossEntropyLoss(ignore_index=5089)

    def validation_step(self, data):
        pass
    def training_step(self, data):
        y_pred = self.model(data)
        y_truth = data["answer"][:, 1:].flatten()

        loss_reconstruction = self.criterion(y_pred[:,:-1].reshape(y_truth.shape[0], -1), y_truth)
        (loss_reconstruction).backward()
        return loss_reconstruction.item()

    def on_validation_end(self, resp):
        pass

    def validation(self, validation_dl):
        pass



    def fit(self,
            training_dl,
            validation_dl,
            learning_rate = 1e-3,
            validation_frequency = 8,
            max_iter = 10000,
            gpu=False,

        ):
        if(gpu):
          self.model = self.model.cuda()
        self.before_training_loop(learning_rate)

        iter_count = 0
        loss_buffer = []
        pbar = trange(max_iter)

        while(True):
            for data in training_dl:
                if gpu:
                    data = {k:v.cuda() for k, v in data.items()}
                self.optimizer.zero_grad()
                loss_buffer += [self.training_step(data)]
                self.optimizer.step()

                if(iter_count  % validation_frequency == 0):
                    print("Loss at iteration %s is %s"%(iter_count, np.mean(loss_buffer)))
                    self.validation(validation_dl)
                    loss_buffer = []

                iter_count += 1
                if(iter_count > max_iter):
                    return
                pbar.update(1)
        return

In [23]:
import os
from torch.utils.data import DataLoader
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
collator = DialogueCollator(mybpe)
training_dl = DataLoader(training_set, batch_size=16, shuffle=True, collate_fn = collator, num_workers=2)
mm = MyModel(embeddings)
my_trainer = Trainer(mm, padding_idx=mybpe.pad_token_id)
my_trainer.fit(training_dl, None, validation_frequency=250, gpu=True, max_iter=20000)

  0%|          | 0/20000 [00:00<?, ?it/s]

Loss at iteration 0 is 10.81336784362793
Loss at iteration 250 is 3.1261091189384462
Loss at iteration 500 is 1.9874086880683899
Loss at iteration 750 is 1.7024827733039856
Loss at iteration 1000 is 1.5765890147686004
Loss at iteration 1250 is 1.4728319737911224
Loss at iteration 1500 is 1.453457531452179


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ace8abeb2e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
if w.is_alive():    
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ace8abeb2e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/

Loss at iteration 1750 is 1.3886805918216705
Loss at iteration 2000 is 1.3211767432689667
Loss at iteration 2250 is 1.2968660731315613
Loss at iteration 2500 is 1.2902998716831207
Loss at iteration 2750 is 1.2442573401927948
Loss at iteration 3000 is 1.2262988247871398
Loss at iteration 3250 is 1.2323648262023925
Loss at iteration 3500 is 1.1977131052017211
Loss at iteration 3750 is 1.1890134935379029
Loss at iteration 4000 is 1.1417381553649901
Loss at iteration 4250 is 1.1276266958713532
Loss at iteration 4500 is 1.1356233828067779
Loss at iteration 4750 is 1.1329971508979797
Loss at iteration 5000 is 1.1218732986450195


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ace8abeb2e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process


...

Loss at iteration 12500 is 0.9690332012176514
Loss at iteration 12750 is 0.9648957884311676
Loss at iteration 13000 is 0.9970598323345184
Loss at iteration 13250 is 0.9865566873550415
Loss at iteration 13500 is 0.9710191345214844
Loss at iteration 13750 is 0.974619866490364
Loss at iteration 14000 is 0.9832273508310319
Loss at iteration 14250 is 0.958823999285698
Loss at iteration 14500 is 0.9371959111690521
Loss at iteration 14750 is 0.925283768415451
Loss at iteration 15000 is 0.9229377664327622
Loss at iteration 15250 is 0.9288491027355195
Loss at iteration 15500 is 0.9226217739582062
Loss at iteration 15750 is 0.9412139401435852


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ace8abeb2e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ace8abeb2e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/

Loss at iteration 16000 is 0.9470832198858261
Loss at iteration 16250 is 0.9362660610675811
Loss at iteration 16500 is 0.9379168539047241
Loss at iteration 16750 is 0.9466201273202897
Loss at iteration 17000 is 0.9469926664829255
Loss at iteration 17250 is 0.9592732955217361
Loss at iteration 17500 is 0.9325463659763337
Loss at iteration 17750 is 0.9511863987445831
Loss at iteration 18000 is 0.9018602299690247
Loss at iteration 18250 is 0.9045651292800904
Loss at iteration 18500 is 0.8991074962615967
Loss at iteration 18750 is 0.8893540341854096
Loss at iteration 19000 is 0.9114233112335205
Loss at iteration 19250 is 0.9107954213619233


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ace8abeb2e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ace8abeb2e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/

Loss at iteration 19500 is 0.8942934281826019
Loss at iteration 19750 is 0.9116609382629395
Loss at iteration 20000 is 0.9152199792861938


In [20]:
training_set[1]

{'history': ['i need a place to dine in the center thats expensive',
  'I have several options for you; do you prefer African, Asian, or British food?',
  'Any sort of food would be fine, as long as it is a bit expensive. Could I get the phone number for your recommendation?'],
 'answer': 'There is an Afrian place named Bedouin in the centre. How does that sound?'}

In [24]:
g =  collator([training_set[1]])
mybpe.decode(mm.cpu().generate(g), skip_sepcial_tokens=True)

" I'm sorry, I don't have any hotels that match your criteria. Would you like to try a different area?[PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]"

In [36]:
class chatbot(object):
  def __init__(self):
    pass

  def answer(self, current_input):
    return "Not Implemented"

  def start(self):
    current_answer = "Start dialogue"
    current_input = ""
    while(current_input != 'exit'):
      current_input = input("Bot: "+current_answer + " \nUser: ")
      current_answer = self.answer(current_input)

class RNNChitChat(Chatbot):
  def __init__(self, model, tokenizer, collator, history_len = 3):
    self.model = model
    self.tokenizer = tokenizer
    self.history_len = history_len
    self.history = []

  def answer(self, current_input):
    self.history.append(current_input)
    self.start_token = 'start'
    data = self.collator({'history': history[-self.history_len], 'answer':''})
    answer = self.tokenizer.decode(self.model.generate(self.tokenizer([data])))
    answer = answer.split('[PAD]')[0]
    self.history.append(answer)
    return answer

  def start(self):
    current_answer = "Start dialogue"
    current_input = ""
    while(current_input != 'exit'):
      current_input = input("Bot: "+current_answer + " \nUser: ")
      current_answer = self.answer(current_input)

In [37]:
my_chatbot = chatbot()
my_chatbot.start()

Bot: Start dialogue 
User: test


KeyboardInterrupt: Interrupted by user

In [15]:
mybpe.decode(g['answer'][0])

'start There is an Afrian place named Bedouin in the centre. How does that sound?'

In [16]:
mybpe.

SyntaxError: invalid syntax (<ipython-input-16-162508b1293d>, line 1)